In [4]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

## 1 Prise en main - lecture vidéo et fichiers images

![image.png](./img/part1.1.png)


In [19]:
import cv2
import time

# Ouvrir la capture vidéo
cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_SETTINGS, 1)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cap.set(cv2.CAP_PROP_GAIN, 0)

# Initialiser la liste des temps de capture
temps_captures = [time.time()]

while True:
    # Capturer une image
    ret, image = cap.read()

    # Afficher l'image RGB
    cv2.imshow('Image RGB', image)

    # Convertir l'image en niveau de gris
    image_grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Afficher l'image en niveau de gris
    cv2.imshow('Image en Niveau de Gris', image_grayscale)

    # Sortir de la boucle si 'q' est pressé
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libérer la capture vidéo
cap.release()

# Fermer toutes les fenêtres OpenCV
cv2.destroyAllWindows()


KeyboardInterrupt: 

### 1.3 Affichage de photos


![image.png](./img/1.3.png)

In [1]:
import cv2
import os

def main():
    # Chemin vers le dossier contenant les images
    dossier_images = "./calib_gopro/calib_gopro/"

    # Lire les noms des fichiers dans le dossier
    for i in range(1, 28):
        # Générer le nom du fichier
        nom_fichier = f"GOPR84{str(i).zfill(2)}.JPG"
        chemin_complet = os.path.join(dossier_images, nom_fichier)

        # Vérifier si le fichier existe
        if os.path.isfile(chemin_complet):
            # Lire l'image
            image = cv2.imread(chemin_complet)

            # Afficher l'image RGB
            cv2.imshow('Image RGB', image)

            # Convertir l'image en niveau de gris
            image_grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # Afficher l'image en niveau de gris
            cv2.imshow('Image en Niveau de Gris', image_grayscale)

            # Attendre une touche pour passer à l'image suivante
            cv2.waitKey(0)
        else:
            print(f"Le fichier {nom_fichier} n'existe pas")

    # Fermer toutes les fenêtres OpenCV
    cv2.destroyAllWindows()

if __name__ == "__main__":main()

## 2 Calibration d’une caméra en OpenCV
### 2.1 Détection d’un échiquier avec les images du dossier

![image.png](./img/2.1.png)

In [1]:
import cv2
import os

def detecter_coins_echiquier(image, board_sz):
    # Convertir l'image en niveaux de gris pour la détection
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Trouver les coins de l'échiquier dans l'image
    found, corners = cv2.findChessboardCorners(gray_image, board_sz, None,
                                               cv2.CALIB_CB_ADAPTIVE_THRESH | cv2.CALIB_CB_FILTER_QUADS)
    return found, corners

def raffiner_coins(gray_image, corners):
    # Critères de raffinement des coins (précision et nombre max d'itérations)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
    # Raffiner la position des coins pour plus de précision
    cv2.cornerSubPix(gray_image, corners, (11, 11), (-1, -1), criteria)
    return corners

def afficher_coins(image, board_sz, corners, found):
    # Dessiner les coins sur l'image si l'échiquier est trouvé
    cv2.drawChessboardCorners(image, board_sz, corners, found)
    # Afficher l'image avec les coins de l'échiquier
    cv2.imshow('Image avec Echiquier', image)
    # Attendre une touche pour continuer
    cv2.waitKey(0)

def main():
    # Chemin vers le dossier des images de l'échiquier
    dossier_images = "./calib_gopro/calib_gopro/"
    # Taille de l'échiquier (nombre de coins horizontaux et verticaux)
    board_sz = (9, 6)

    # Parcourir les images dans le dossier
    for i in range(1, 28):
        nom_fichier = f"GOPR84{str(i).zfill(2)}.JPG"
        chemin_complet = os.path.join(dossier_images, nom_fichier)

        # Vérifier si l'image existe et la traiter
        if os.path.isfile(chemin_complet):
            image = cv2.imread(chemin_complet)
            # Détecter les coins de l'échiquier dans l'image
            found, corners = detecter_coins_echiquier(image, board_sz)

            # Si l'échiquier est trouvé, raffiner et afficher les coins
            if found:
                corners = raffiner_coins(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), corners)
                afficher_coins(image, board_sz, corners, found)

    # Fermer toutes les fenêtres à la fin du traitementq
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

### 2.1 Détection d’un échiquier avec la caméra

![image.png](./img/2.1bis.png)

In [1]:
import cv2
import time

def detecter_coins_echiquier(image, board_sz):
    # Convertir l'image en niveaux de gris pour la détection
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Trouver les coins de l'échiquier
    found, corners = cv2.findChessboardCorners(gray_image, board_sz, None,
                                               cv2.CALIB_CB_ADAPTIVE_THRESH | cv2.CALIB_CB_FILTER_QUADS)
    return found, corners

def raffiner_coins(gray_image, corners):
    # Critères de raffinement des coins (précision et nombre max d'itérations)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
    # Raffiner la position des coins
    cv2.cornerSubPix(gray_image, corners, (11, 11), (-1, -1), criteria)
    return corners

def afficher_coins(image, board_sz, corners, found):
    # Dessiner les coins sur l'image si l'échiquier est trouvé
    if found:
        cv2.drawChessboardCorners(image, board_sz, corners, found)
    # Afficher l'image avec ou sans les coins de l'échiquier
    cv2.imshow('Image avec Echiquier', image)

def main():
    # Configuration de la capture vidéo
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_SETTINGS, 1)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    cap.set(cv2.CAP_PROP_GAIN, 0)
    # Dimensions de l'échiquier (nombre de coins horizontaux et verticaux)
    board_sz = (9, 6)

    while True:
        ret, image = cap.read()
        if not ret:
            break

        # Tentative de détection des coins de l'échiquier
        found, corners = detecter_coins_echiquier(image, board_sz)

        # Raffiner et afficher les coins si détectés, sinon afficher l'image telle quelle
        if found:
            corners = raffiner_coins(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), corners)
        afficher_coins(image, board_sz, corners, found)

        # Sortir de la boucle si 'q' est pressé
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Libérer le flux vidéo et fermer toutes les fenêtres
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

KeyboardInterrupt: 

## 2.2 Calibration de la caméra

![image.png](./img/2.2.png)

In [2]:
import cv2
import numpy as np

def detecter_coins_echiquier(image, board_sz):
    # Convertir l'image en niveaux de gris pour la détection
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Trouver les coins de l'échiquier
    found, corners = cv2.findChessboardCorners(gray_image, board_sz, None,
                                               cv2.CALIB_CB_ADAPTIVE_THRESH | cv2.CALIB_CB_FILTER_QUADS)
    return found, corners

def raffiner_coins(gray_image, corners):
    # Critères de raffinement des coins (précision et nombre max d'itérations)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
    # Raffiner la position des coins
    cv2.cornerSubPix(gray_image, corners, (11, 11), (-1, -1), criteria)
    return corners

def afficher_coins(image, board_sz, corners, found):
    # Dessiner les coins sur l'image si l'échiquier est trouvé
    if found:
        cv2.drawChessboardCorners(image, board_sz, corners, found)
    # Afficher l'image avec ou sans les coins de l'échiquier
    cv2.imshow('Image avec Echiquier', image)

def creer_points_objet(board_sz, square_size):
    # Création des points objet (coordonnées 3D des coins de l'échiquier)
    objp = np.zeros((board_sz[0] * board_sz[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:board_sz[0], 0:board_sz[1]].T.reshape(-1, 2) * square_size
    return objp

def main():
    # Configuration de la capture vidéo
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    # Dimensions de l'échiquier (nombre de coins horizontaux et verticaux)
    board_sz = (9, 6)
    square_size = 1.0  # Taille d'un carré sur l'échiquier
    objp = creer_points_objet(board_sz, square_size)

    # Stockage des points objet et image pour chaque image où l'échiquier est détecté
    object_points = []  # Points 3D dans le monde réel
    image_points = []   # Points 2D dans l'image plane

    while True:
        ret, image = cap.read()
        if not ret:
            break

        found, corners = detecter_coins_echiquier(image, board_sz)
        if found:
            corners_refined = raffiner_coins(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), corners)
            image_points.append(corners_refined)
            object_points.append(objp)
            afficher_coins(image, board_sz, corners_refined, found)
        else:
            cv2.imshow('Image avec Echiquier', image)

        if len(image_points) >= 10:  # Nombre suffisant d'images pour la calibration
            ret, intrinsic, distCoeffs, rvecs, tvecs = cv2.calibrateCamera(object_points, image_points, 
                                                                          (image.shape[1], image.shape[0]), None, None)
            print("Matrice intrinsèque:\n", intrinsic)
            print("Coefficients de distorsion:\n", distCoeffs)
            # break

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Matrice intrinsèque:
 [[392.29405185   0.         316.9585197 ]
 [  0.         391.84250727 167.07962184]
 [  0.           0.           1.        ]]
Coefficients de distorsion:
 [[ 0.08032504  0.31481303 -0.00534152 -0.00852357 -1.89060528]]
Matrice intrinsèque:
 [[400.37677884   0.         317.26700997]
 [  0.         399.70929562 168.48918004]
 [  0.           0.           1.        ]]
Coefficients de distorsion:
 [[ 0.07273432  0.45412962 -0.00466723 -0.00847726 -2.52698473]]
Matrice intrinsèque:
 [[408.43611818   0.         317.58277983]
 [  0.         407.83671148 168.97827119]
 [  0.           0.           1.        ]]
Coefficients de distorsion:
 [[ 0.09276709  0.3823018  -0.00559083 -0.00664482 -2.61189187]]
Matrice intrinsèque:
 [[421.78277847   0.         317.05031112]
 [  0.         421.81966056 169.32076919]
 [  0.           0.           1.        ]]
Coefficients de distorsion:
 [[ 0.1391208   0.20530498 -0.00773098 -0.00383744 -2.68603826]]
Matrice intrinsèque:
 [[424.7886

## 2.3 Redressement de l’image

![image.png](./img/2.3.png)

In [5]:
import cv2
import numpy as np

def detecter_coins_echiquier(image, board_sz):
    # Convertir l'image en niveaux de gris pour la détection
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Trouver les coins de l'échiquier
    found, corners = cv2.findChessboardCorners(gray_image, board_sz, None,
                                               cv2.CALIB_CB_ADAPTIVE_THRESH | cv2.CALIB_CB_FILTER_QUADS)
    return found, corners

def raffiner_coins(gray_image, corners):
    # Critères de raffinement des coins (précision et nombre max d'itérations)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
    # Raffiner la position des coins
    cv2.cornerSubPix(gray_image, corners, (11, 11), (-1, -1), criteria)
    return corners

def afficher_coins(image, board_sz, corners, found):
    # Dessiner les coins sur l'image si l'échiquier est trouvé
    if found:
        cv2.drawChessboardCorners(image, board_sz, corners, found)
    # Afficher l'image avec ou sans les coins de l'échiquier
    cv2.imshow('Image avec Echiquier', image)

def creer_points_objet(board_sz, square_size):
    # Création des points objet (coordonnées 3D des coins de l'échiquier)
    objp = np.zeros((board_sz[0] * board_sz[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:board_sz[0], 0:board_sz[1]].T.reshape(-1, 2) * square_size
    return objp

def main():
    # Configuration de la capture vidéo
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    # Dimensions de l'échiquier (nombre de coins horizontaux et verticaux)
    board_sz = (9, 6)
    square_size = 1.0  # Taille d'un carré sur l'échiquier
    objp = creer_points_objet(board_sz, square_size)

    # Stockage des points objet et image pour chaque image où l'échiquier est détecté
    object_points = []  # Points 3D dans le monde réel
    image_points = []   # Points 2D dans l'image plane

    while True:
        ret, image = cap.read()
        if not ret:
            break

        found, corners = detecter_coins_echiquier(image, board_sz)
        if found:
            corners_refined = raffiner_coins(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), corners)
            image_points.append(corners_refined)
            object_points.append(objp)
            afficher_coins(image, board_sz, corners_refined, found)
        else:
            cv2.imshow('Image avec Echiquier', image)

        if len(image_points) >= 10:  # Nombre suffisant d'images pour la calibration
            ret, intrinsic, distCoeffs, rvecs, tvecs = cv2.calibrateCamera(object_points, image_points, (image.shape[1], image.shape[0]), None, None)
            # print("Matrice intrinsèque:\n", intrinsic)
            # print("Coefficients de distorsion:\n", distCoeffs)

            # Redressement de l'image
            image_undistorted = np.zeros_like(image)
            image_undistorted = cv2.undistort(image, intrinsic, distCoeffs, None, intrinsic)
            
            # Afficher l'image redressée
            cv2.imshow('Image Redressée', image_undistorted)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


KeyboardInterrupt: 

## 2.4 Mise en forme de votre code

![image.png](./img/2.4.png)

In [8]:
import cv2
import numpy as np

def demander_informations_utilisateur():
    camera_id = int(input("Entrez l'identifiant de la caméra utilisée (par exemple, 0 pour la caméra par défaut): "))
    board_width = int(input("Entrez le nombre de coins intérieurs à l'échiquier dans la largeur : "))
    board_height = int(input("Entrez le nombre de coins intérieurs à l'échiquier dans la hauteur : "))
    num_images = int(input("Entrez le nombre d'images à utiliser pour la calibration : "))
    
    return camera_id, (board_width, board_height), num_images

def detecter_coins_echiquier(image, board_sz):
    # Convertir l'image en niveaux de gris pour la détection
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Trouver les coins de l'échiquier
    found, corners = cv2.findChessboardCorners(gray_image, board_sz, None,
                                               cv2.CALIB_CB_ADAPTIVE_THRESH | cv2.CALIB_CB_FILTER_QUADS)
    return found, corners

def raffiner_coins(gray_image, corners):
    # Critères de raffinement des coins (précision et nombre max d'itérations)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
    # Raffiner la position des coins
    cv2.cornerSubPix(gray_image, corners, (11, 11), (-1, -1), criteria)
    return corners

def afficher_coins(image, board_sz, corners, found):
    # Dessiner les coins sur l'image si l'échiquier est trouvé
    if found:
        cv2.drawChessboardCorners(image, board_sz, corners, found)
    # Afficher l'image avec ou sans les coins de l'échiquier
    cv2.imshow('Image avec Echiquier', image)

def creer_points_objet(board_sz, square_size):
    # Création des points objet (coordonnées 3D des coins de l'échiquier)
    objp = np.zeros((board_sz[0] * board_sz[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:board_sz[0], 0:board_sz[1]].T.reshape(-1, 2) * square_size
    return objp

def main():
    camera_id, board_sz, num_images = demander_informations_utilisateur()

    # Configuration de la capture vidéo
    cap = cv2.VideoCapture(camera_id)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    square_size = 1.0  # Taille d'un carré sur l'échiquier
    objp = creer_points_objet(board_sz, square_size)

    # Stockage des points objet et image pour chaque image où l'échiquier est détecté
    object_points = []  # Points 3D dans le monde réel
    image_points = []   # Points 2D dans l'image plane
    images_used = 0

    while images_used < num_images:
        ret, image = cap.read()
        if not ret:
            break

        found, corners = detecter_coins_echiquier(image, board_sz)
        if found:
            corners_refined = raffiner_coins(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), corners)
            image_points.append(corners_refined)
            object_points.append(objp)
            afficher_coins(image, board_sz, corners_refined, found)
            images_used += 1
        else:
            cv2.imshow('Image avec Echiquier', image)

        key = cv2.waitKey(1)
        if key == 27:  # Touche 'ESC' pour passer à l'étape suivante
            break

    # Calcul de la matrice intrinsèque et des coefficients de distorsion
    ret, intrinsic, distCoeffs, rvecs, tvecs = cv2.calibrateCamera(
        object_points, image_points, (image.shape[1], image.shape[0]), None, None
    )

    print("Matrice intrinsèque:\n", intrinsic)
    print("Coefficients de distorsion:\n", distCoeffs)

    # Boucle d'affichage du résultat
    while True:
        ret, image = cap.read()
        if not ret:
            break

        # Redressement de l'image
        image_undistorted = cv2.undistort(image, intrinsic, distCoeffs, None, intrinsic)

        # Afficher l'image redressée ou l'image originale
        cv2.imshow('Image', image_undistorted  )

        if cv2.waitKey(1) == 27:  # Touche 'ESC' pour quitter
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


KeyboardInterrupt: 

# 3 Calibration d’une caméra en OpenCV à partir d’un ensemble d’images

![image.png](../Lab3/img/part3.png)

In [17]:
import cv2
import numpy as np
import os

def demander_informations_utilisateur():
    board_width = int(input("Entrez le nombre de coins intérieurs à l'échiquier dans la largeur : "))
    board_height = int(input("Entrez le nombre de coins intérieurs à l'échiquier dans la hauteur : "))
    num_images = int(input("Entrez le nombre d'images à utiliser pour la calibration : "))
    
    return board_width, board_height, num_images

def charger_images_calib_gopro(dossier):
    images = []
    for filename in os.listdir(dossier):
        chemin_image = os.path.join(dossier, filename)
        if os.path.isfile(chemin_image) and filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            images.append(cv2.imread(chemin_image))
    return images

def detecter_coins_echiquier(image, board_sz):
    # Convertir l'image en niveaux de gris pour la détection
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Trouver les coins de l'échiquier
    found, corners = cv2.findChessboardCorners(gray_image, board_sz, None, cv2.CALIB_CB_ADAPTIVE_THRESH | cv2.CALIB_CB_FILTER_QUADS)
    return found, corners

def raffiner_coins(gray_image, corners):
    # Critères de raffinement des coins (précision et nombre max d'itérations)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
    # Raffiner la position des coins
    cv2.cornerSubPix(gray_image, corners, (11, 11), (-1, -1), criteria)
    return corners

def afficher_coins(image, board_sz, corners, found):
    # Dessiner les coins sur l'image si l'échiquier est trouvé
    if found:
        cv2.drawChessboardCorners(image, board_sz, corners, found)
    # Afficher l'image avec ou sans les coins de l'échiquier
    cv2.imshow('Image avec Echiquier', image)

def creer_points_objet(board_sz, square_size):
    # Création des points objet (coordonnées 3D des coins de l'échiquier)
    objp = np.zeros((board_sz[0] * board_sz[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:board_sz[0], 0:board_sz[1]].T.reshape(-1, 2) * square_size
    return objp

def main():
    board_width, board_height, num_images = demander_informations_utilisateur()

    # Chargement des images de calibration depuis le dossier "calib_gopro"
    dossier_calib_gopro = "./calib_gopro/calib_gopro/"
    images_calib_gopro = charger_images_calib_gopro(dossier_calib_gopro)

    # Vérifier si suffisamment d'images sont disponibles
    if len(images_calib_gopro) < num_images:
        print(f"Erreur: Seules {len(images_calib_gopro)} images sont disponibles dans le dossier {dossier_calib_gopro}.")
        return

    # Dimensions de l'échiquier (nombre de coins horizontaux et verticaux)
    board_sz = (board_width, board_height)
    square_size = 1.0  # Taille d'un carré sur l'échiquier
    objp = creer_points_objet(board_sz, square_size)

    # Stockage des points objet et image pour chaque image où l'échiquier est détecté
    object_points = []  # Points 3D dans le monde réel
    image_points = []   # Points 2D dans l'image plane
    images_used = 0

    while images_used < num_images:
        image = images_calib_gopro[images_used]
        found, corners = detecter_coins_echiquier(image, board_sz)
        if found:
            corners_refined = raffiner_coins(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), corners)
            image_points.append(corners_refined)
            object_points.append(objp)
            afficher_coins(image, board_sz, corners_refined, found)
            images_used += 1
        else:
            cv2.imshow('Image avec Echiquier', image)

        key = cv2.waitKey(0)
        if key == 27:  # Touche 'ESC' pour passer à l'étape suivante
            break

    # Calcul de la matrice intrinsèque et des coefficients de distorsion
    ret, intrinsic, distCoeffs, rvecs, tvecs = cv2.calibrateCamera(
        object_points, image_points, (image.shape[1], image.shape[0]), None, None
    )

    print("Matrice intrinsèque:\n", intrinsic)
    print("Coefficients de distorsion:\n", distCoeffs)

    # Boucle d'affichage du résultat
    for image in images_calib_gopro:
        # Redressement de l'image
        image_undistorted = cv2.undistort(image, intrinsic, distCoeffs, None, intrinsic)

        # Afficher l'image redressée ou l'image originale
        cv2.imshow('Image', image_undistorted if cv2.waitKey(0) == ord('r') else image)

        if cv2.waitKey(0) == 27:  # Touche 'ESC' pour quitter
            break

    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Matrice intrinsèque:
 [[275.48989976   0.         322.7617264 ]
 [  0.         274.72955319 228.81551489]
 [  0.           0.           1.        ]]
Coefficients de distorsion:
 [[-2.98077829e-01  1.01979313e-01 -1.66867808e-04  7.14343987e-04
  -1.34974327e-02]]


KeyboardInterrupt: 